In [69]:
# extract road / landuse / station features from OSM

In [1]:
import geojson
import geopandas as gpd
import pandas as pd
import folium
import utm
from shapely import geometry
from shapely import ops
import numpy as np
import os
import data_process as dp
import warnings
warnings. filterwarnings("ignore")

## Load geojson map and extract road from OSM

In [2]:
def utm_convert(row):
    ls = []
    for y in row['location']:
        ls.append(list(utm.from_latlon(y[1],y[0])[0:2]))
        # convert location information in latitude and longitude to Universal Transverse Mercator coordinate system (UTM). 
        # This is especially useful for large dense arrays in a small area
    return ls 

In [3]:
def linestring(row):
    return geometry.LineString(row.location)

In [4]:
def polygon(row):
    try: 
        temp = geometry.Polygon(row.location)
        return temp
    except:
        return np.nan

## Extract station information

In [492]:
# Path to relevant geojson map of OSM features
# Edinburgh

osmStation = []

path = './data/geojson/trainingmap.geojson'
with open(path, encoding="utf-8") as f:
    osmlines = geojson.load(f)
for allFeatures in osmlines.features:
    if 'highway' in allFeatures['properties']:
        if allFeatures["properties"]["highway"] == "bus_stop":
            roadinfo = allFeatures['properties']
            roadinfo['station'] = 'bus_stop'
            locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
            if locarr.ndim == 3:
                locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
            if locarr.ndim == 1:
                locarr = np.array([locarr,locarr]) 
            roadinfo['location'] = locarr
            osmStation.append(roadinfo)

    if 'railway' in allFeatures['properties']:
        if allFeatures["properties"]["railway"] == "station" or allFeatures["properties"]["railway"] == 'halt' \
        or allFeatures["properties"]["railway"] == "platform" or allFeatures["properties"]["railway"] == "stop":
            roadinfo = allFeatures['properties']
            roadinfo['station'] = 'railway_station'
            locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
            if locarr.ndim == 3:
                locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
            if locarr.ndim == 1:
                locarr = np.array([locarr,locarr]) 
            roadinfo['location'] = locarr
            osmStation.append(roadinfo)
            
        elif allFeatures["properties"]["railway"] == "subway_entrance":
            roadinfo = allFeatures['properties']
            roadinfo['station'] = 'subway_station'
            locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
            if locarr.ndim == 3:
                locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
            if locarr.ndim == 1:
                locarr = np.array([locarr,locarr]) 
            roadinfo['location'] = locarr
            osmStation.append(roadinfo)

    if 'station' in allFeatures['properties']:
        if allFeatures["properties"]["station"] == 'subway':
            roadinfo = allFeatures['properties']
            roadinfo['station'] = 'subway_station'
            locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
            if locarr.ndim == 3:
                locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
            if locarr.ndim == 1:
                locarr = np.array([locarr,locarr]) 
            roadinfo['location'] = locarr
            osmStation.append(roadinfo)
                       
osmStation = pd.DataFrame.from_dict(osmStation)
osmStation = osmStation.set_index('id')

In [375]:
# Path to relevant geojson map of OSM features
# London / Delhi

osmStation = []
for i in range(1,6):
    path = './data/geojson/toby_new/map{}.geojson'.format(i) 
    with open(path, encoding="utf-8") as f:
        osmlines = geojson.load(f)
    for allFeatures in osmlines.features:
        if 'highway' in allFeatures['properties']:
            if allFeatures["properties"]["highway"] == "bus_stop":
                roadinfo = allFeatures['properties']
                roadinfo['station'] = 'bus_stop'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
                if locarr.ndim == 3:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr]) 
                roadinfo['location'] = locarr
                osmStation.append(roadinfo)

        if 'railway' in allFeatures['properties']:
            if allFeatures["properties"]["railway"] == "station" or allFeatures["properties"]["railway"] == 'halt' \
            or allFeatures["properties"]["railway"] == "platform" or allFeatures["properties"]["railway"] == "stop":
                roadinfo = allFeatures['properties']
                roadinfo['station'] = 'railway_station'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
                if locarr.ndim == 3:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr]) 
                roadinfo['location'] = locarr
                osmStation.append(roadinfo)
            
            elif allFeatures["properties"]["railway"] == "subway_entrance":
                roadinfo = allFeatures['properties']
                roadinfo['station'] = 'subway_station'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
                if locarr.ndim == 3:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr]) 
                roadinfo['location'] = locarr
                osmStation.append(roadinfo)

        if 'station' in allFeatures['properties']:
            if allFeatures["properties"]["station"] == 'subway':
                roadinfo = allFeatures['properties']
                roadinfo['station'] = 'subway_station'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
                if locarr.ndim == 3:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr]) 
                roadinfo['location'] = locarr
                osmStation.append(roadinfo)
                       
osmStation = pd.DataFrame.from_dict(osmStation)
osmStation = osmStation.set_index('osm_id')

In [493]:
osmStation.shape

(1729, 84)

## Add station info to dataset

In [494]:
np.unique(osmStation['station'].astype(str), return_counts=True)

(array(['bus_stop', 'railway_station'], dtype=object), array([1620,  109]))

In [495]:
# Filtering and pre-processing 
# osmRoads = osmRoads[osmRoads['type'] != 'multipolygon']
osmStation['utmLocation'] = osmStation.apply(lambda row: utm_convert(row), axis=1) # convert to utm coordinate
osmStation['locationLineString'] = osmStation.apply(lambda row: linestring(row), axis=1)
# construct the line using a list of coordinate-tuples
osmStation = osmStation.filter(['name','station', 'location', 'utmLocation', 'locationLineString'])
# only care about related attributes 

In [496]:
osm_station = gpd.GeoDataFrame(osmStation)
osm_station = osm_station.rename({'locationLineString': 'geometry'}, axis = 1)
osm_station['geometry'] = gpd.GeoSeries(osm_station['geometry'])
osm_station.sindex

rtree.index.Index(bounds=[-3.419375, 53.2677, -2.329946, 55.990886], size=1729)

In [497]:
data_dir = "./data/indoor_outdoor"
data_path = os.path.join(data_dir, "new_data_set.csv")
#data_path = os.path.join(data_dir, "london_unseen_data.csv")
#data_path = os.path.join(data_dir, "all_data_new_select.csv")
full_data = pd.read_csv(data_path)

In [498]:
def min_max_norm(data, col):
    target_col = data[col]
    max_num = max(target_col.dropna())
    min_num = min(target_col.dropna())
    std = (target_col - min_num) / (max_num - min_num)
    data[col] = std
    
    return data

In [499]:
full_data = min_max_norm(full_data, 'temperature')
full_data = min_max_norm(full_data, 'humidity')

In [500]:
# Now convert this to a points geoDataFrame
points_full = gpd.GeoDataFrame(full_data, geometry = gpd.points_from_xy(full_data['gpsLongitude'],full_data['gpsLatitude']))

In [501]:
offset = 0.00025 # Roughly 50 meters
bbox_full = points_full.bounds + [-offset, -offset, offset, offset]

In [502]:
hits_full = bbox_full.apply(lambda row: list(osm_station.sindex.intersection(row)), axis=1)

In [503]:
dist_df_full = pd.DataFrame({'pt_idx':np.repeat(hits_full.index, hits_full.apply(len)), 'close_station_idx': np.concatenate(hits_full.values)})
dist_df_full = dist_df_full.join(points_full['geometry'].rename('point'), on='pt_idx')
dist_df_full = dist_df_full.join(osm_station[['geometry','location', 'station']].reset_index(drop=True), on='close_station_idx')

dist_gdf_full = gpd.GeoDataFrame(dist_df_full)
dist_gdf_full['station_distance'] = dist_gdf_full['geometry'].distance(gpd.GeoSeries(dist_gdf_full['point']))
dist_gdf_full = dist_gdf_full.sort_values(by=['station_distance'])
dist_gdf_full = dist_gdf_full.groupby('pt_idx').first()

In [504]:
new_full_data = full_data.join(dist_gdf_full[['station', 'station_distance', 'close_station_idx']])
new_full_data['station'] = new_full_data['station'].fillna('none')
new_full_data['station_distance'] = new_full_data['station_distance'].fillna(0.010)
np.unique(new_full_data['station'].astype(str), return_counts=True)

(array(['bus_stop', 'none', 'railway_station'], dtype=object),
 array([ 538, 3947,   66]))

In [505]:
new_full_data.shape

(4551, 33)

In [506]:
new_full_data.to_csv(os.path.join(data_dir,'new_data_set_with_station.csv'))
#new_full_data.to_csv(os.path.join(data_dir,'london_unseen_data_with_station.csv'))
#new_full_data.to_csv(os.path.join(data_dir,'all_data_new_select_with_station.csv'))

## Process OSM road feature

In [538]:
# Path to relevant geojson map of OSM features
# Edinburgh

osmRoads = []

path = './data/geojson/trainingmap.geojson'
with open(path, encoding="utf-8") as f:
    osmlines = geojson.load(f)
for allFeatures in osmlines.features:
    if 'highway' in allFeatures['properties']:
        roadinfo = allFeatures['properties']
        
        locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
        if locarr.ndim == 3:
            locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
        
        if locarr.ndim == 1:
            locarr = np.array([locarr,locarr]) 
        
        roadinfo['location'] = locarr
        
        osmRoads.append(roadinfo)
osmRoads = pd.DataFrame.from_dict(osmRoads)
osmRoads = osmRoads.set_index('id')

In [5]:
# Path to relevant geojson map of OSM features  
# London / Delhi
osmRoads = []

for i in range(1,6):
    path = './data/geojson/toby_new/map{}.geojson'.format(i) 
    with open(path, encoding="utf-8") as f:
        osmlines = geojson.load(f)
    for allFeatures in osmlines.features:
        if 'highway' in allFeatures['properties']:
            roadinfo = allFeatures['properties']
        
            locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
        
            if locarr.ndim == 3:
                locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim
        
            if locarr.ndim == 1:
                locarr = np.array([locarr,locarr]) 
        
            roadinfo['location'] = locarr
        
            osmRoads.append(roadinfo)
osmRoads = pd.DataFrame.from_dict(osmRoads)
osmRoads = osmRoads.set_index('osm_id')

In [37]:
osmRoads.shape

(229403, 5)

## Process highway type

In [38]:
np.unique(osmRoads['highway'].astype(str), return_counts=True)

(array(['footway', 'motorway', 'primary_secondary', 'residential',
        'tertiary', 'unclassified', 'unknown'], dtype=object),
 array([76938,   180, 11937, 79825,  5204,  6435, 48884]))

In [8]:
# Elements in osmRoad: 33757
# bus_stop: 1620; crossing: 891; cycleway: 552; footway: 8053; give_way: 1704; path: 1063; pedestrian: 235; 
# primary: 662; residential: 4804; service: 8588; steps: 1753; tertiary: 1042; track: 414; unclassified: 802;

In [10]:
# Filtering and pre-processing 
#osmRoads = osmRoads[osmRoads['type'] != 'multipolygon']
osmRoads['utmLocation'] = osmRoads.apply(lambda row: utm_convert(row), axis=1) # convert to utm coordinate
osmRoads['locationLineString'] = osmRoads.apply(lambda row: linestring(row), axis=1)
# construct the line using a list of coordinate-tuples
osmRoads = osmRoads.filter(['name','highway', 'location', 'utmLocation', 'locationLineString'])
# only care about related attributes 

In [11]:
for i in range(len(osmRoads)):
    road = osmRoads.iloc[i]
    
    if road['highway'] == 'motorway' or road['highway'] == 'motorway_junction' or road['highway'] == 'motorway_link':
        osmRoads.iloc[i]['highway'] = 'motorway'
    elif road['highway'] == 'primary' or road['highway'] == 'primary_link':
        osmRoads.iloc[i]['highway'] = 'primary_secondary'
    elif road['highway'] == 'secondary' or road['highway'] == 'secondary_link':
        osmRoads.iloc[i]['highway'] = 'primary_secondary'
    elif road['highway'] == 'tertiary' or road['highway'] == 'tertiary_link':
        osmRoads.iloc[i]['highway'] = 'tertiary'
    elif road['highway'] == 'residential' or road['highway'] == 'living_street' or road['highway'] == 'service':
        osmRoads.iloc[i]['highway'] = 'residential'
    elif road['highway'] == 'footway' or road['highway'] == 'cycleway' or road['highway'] == 'pedestrian' \
    or road['highway'] == 'path' or road['highway'] == 'steps':
        osmRoads.iloc[i]['highway'] = 'footway'
    elif road['highway'] == 'unclassified':
        osmRoads.iloc[i]['highway'] = 'unclassified'                                                       
    else:
        osmRoads.iloc[i]['highway'] = 'unknown'

In [ ]:
## only split into residential (residential + footway) & others
for i in range(len(osmRoads)):
    road = osmRoads.iloc[i]
    
    if road['highway'] == 'motorway' or road['highway'] == 'motorway_junction' or road['highway'] == 'motorway_link':
        osmRoads.iloc[i]['highway'] = 'others'
    elif road['highway'] == 'primary' or road['highway'] == 'primary_link':
        osmRoads.iloc[i]['highway'] = 'others'
    elif road['highway'] == 'secondary' or road['highway'] == 'secondary_link':
        osmRoads.iloc[i]['highway'] = 'others'
    elif road['highway'] == 'tertiary' or road['highway'] == 'tertiary_link':
        osmRoads.iloc[i]['highway'] = 'others'
    elif road['highway'] == 'residential' or road['highway'] == 'living_street' or road['highway'] == 'service':
        osmRoads.iloc[i]['highway'] = 'residential'
    elif road['highway'] == 'footway' or road['highway'] == 'cycleway' or road['highway'] == 'pedestrian' \
    or road['highway'] == 'path' or road['highway'] == 'steps':
        osmRoads.iloc[i]['highway'] = 'residential'
    elif road['highway'] == 'unclassified':
        osmRoads.iloc[i]['highway'] = 'others'                                                       
    else:
        osmRoads.iloc[i]['highway'] = 'others'

In [39]:
np.unique(osmRoads['highway'].astype(str), return_counts=True)

(array(['footway', 'motorway', 'primary_secondary', 'residential',
        'tertiary', 'unclassified', 'unknown'], dtype=object),
 array([76938,   180, 11937, 79825,  5204,  6435, 48884]))

## Find the closest road to each point in the training data

1. Find the nearest line to the original location
2. Find the road type

In [40]:
# Generate Spacial index, first need to convert Dataframe to GeoDataFrame, and series to GeoSeries.
# extends popular data science library pandas by adding support for geospatial data
# Each GeoSeries can contain any geometry type and has a GeoSeries.crs attribute, 
# which stores information on the projection (CRS stands for Coordinate Reference System)
osm_roads = gpd.GeoDataFrame(osmRoads)
osm_roads = osm_roads.rename({'locationLineString': 'geometry'}, axis = 1)
osm_roads['geometry'] = gpd.GeoSeries(osm_roads['geometry'])
osm_roads.sindex

rtree.index.Index(bounds=[-0.405502, 51.428063, 0.015316, 51.684585], size=229403)

In [ ]:
osm_roads.head()

## Load training data

In [ ]:
data_dir = "./data/indoor_outdoor"
#data_path = os.path.join(data_dir, "new_data_set_with_station.csv")
#data_path = os.path.join(data_dir, "london_unseen_data.csv")
data_path = os.path.join(data_dir, "all_data_new_select.csv")
full_data = pd.read_csv(data_path)
full_data.head()

In [43]:
# Now convert this to a points geoDataFrame
points_full = gpd.GeoDataFrame(full_data, geometry = gpd.points_from_xy(full_data['gpsLongitude'],full_data['gpsLatitude']))
points_full.head()

,Unnamed: 0,phoneTimestamp,pm1,pm2_5,pm10,temperature,humidity,bin0,bin1,bin2,...,subjectID,location,activity,location_type,activity_type,recording_id,timestamp,tubeline,notes,geometry
0,9076,1614763472266,31.357779,68.628010,168.52596,25.9,38.300000,7102.0,1098.0,248.0,...,XXT012,Indoor,Sitting,50,0,147,2021-03-03 09:24:32.266,N/a,N/a,POINT (-0.13229 51.56523)
1,9077,1614763506447,31.063381,68.659660,199.63565,25.9,39.000000,8090.0,996.0,264.0,...,XXT012,Indoor,Sitting,50,0,147,2021-03-03 09:25:06.447,N/a,N/a,POINT (-0.13215 51.56509)
2,9078,1614763537409,31.755910,68.593600,211.94217,26.7,37.200000,6981.0,1140.0,237.0,...,XXT012,Indoor,Sitting,50,0,147,2021-03-03 09:25:37.409,N/a,N/a,POINT (-0.13219 51.56506)
3,9079,1614763569653,31.561163,71.432800,229.19894,27.6,36.100002,7352.0,1043.0,243.0,...,XXT012,Indoor,Sitting,50,0,147,2021-03-03 09:26:09.653,N/a,N/a,POINT (-0.13219 51.56506)
4,9080,1614763599327,32.124138,66.662285,185.78313,28.2,34.900000,7345.0,977.0,239.0,...,XXT012,Indoor,Sitting,50,0,147,2021-03-03 09:26:39.327,N/a,N/a,POINT (-0.13219 51.56506)


In [44]:
offset = 0.00025 # Roughly 50 meters
bbox_full = points_full.bounds + [-offset, -offset, offset, offset]
bbox_full.head()

,minx,miny,maxx,maxy
0,-0.132540,51.564977,-0.132040,51.565477
1,-0.132398,51.564841,-0.131898,51.565341
2,-0.132443,51.564807,-0.131943,51.565307
3,-0.132443,51.564807,-0.131943,51.565307
4,-0.132443,51.564807,-0.131943,51.565307


In [45]:
hits_full = bbox_full.apply(lambda row: list(osm_roads.sindex.intersection(row)), axis=1)
hits_full

0                [106455, 50749, 49222]
1                       [106455, 50749]
2                       [106455, 50749]
3                       [106455, 50749]
4                       [106455, 50749]
                     ...               
7295     [200688, 54923, 158084, 54924]
7296      [3009, 152931, 200688, 54925]
7297       [152931, 3591, 3010, 200688]
7298    [152931, 200688, 54921, 115413]
7299            [152931, 62529, 191922]
Length: 7300, dtype: object

## Add road type feature to data set

In [46]:
dist_df_full = pd.DataFrame({'pt_idx':np.repeat(hits_full.index, hits_full.apply(len)), 'close_road_idx': np.concatenate(hits_full.values)})
dist_df_full = dist_df_full.join(points_full['geometry'].rename('point'), on='pt_idx')
dist_df_full = dist_df_full.join(osm_roads[['geometry','location', 'highway']].reset_index(drop=True), on='close_road_idx')

dist_gdf_full = gpd.GeoDataFrame(dist_df_full)
dist_gdf_full['distance'] = dist_gdf_full['geometry'].distance(gpd.GeoSeries(dist_gdf_full['point']))
dist_gdf_full = dist_gdf_full.sort_values(by=['distance'])
dist_gdf_full = dist_gdf_full.groupby('pt_idx').first()

In [ ]:
dist_gdf_full.head()

In [48]:
new_full_data = full_data.join(dist_gdf_full[['highway', 'distance', 'close_road_idx']])
new_full_data['highway'] = new_full_data['highway'].fillna('others')
new_full_data['distance'] = new_full_data['distance'].fillna(0.005)
np.unique(new_full_data['highway'].astype(str), return_counts=True)

(array(['footway', 'others', 'primary_secondary', 'residential',
        'tertiary', 'unclassified', 'unknown'], dtype=object),
 array([ 809,  106,  957, 3706,  118,  142, 1462]))

In [ ]:
new_full_data.head()

In [50]:
#new_full_data.to_csv(os.path.join(data_dir,'new_data_set_with_station_roads_distance.csv'))
#new_full_data.to_csv(os.path.join(data_dir,'london_unseen_data_with_roads_distance.csv'))
new_full_data.to_csv(os.path.join(data_dir,'all_data_new_select_with_roads_distance.csv'))

## Extract landuse from OSM

In [571]:
# path = './data/geojson/unseen_central_london.geojson'
# Edinburgh

osmLands = []
for i in range(1,2):
    path = './data/geojson/trainingmap.geojson'
    with open(path, encoding="utf-8") as f:
        osmlines = geojson.load(f)
    for allFeatures in osmlines.features:
        if 'landuse' in allFeatures['properties']:
            ## OUTLAND
            if allFeatures["properties"]["landuse"] == "grass" or allFeatures["properties"]["landuse"] == "basin" \
            or allFeatures["properties"]["landuse"] == "forest" or allFeatures["properties"]["landuse"] == "greenfield" \
            or allFeatures["properties"]["landuse"] == "meadow" or allFeatures["properties"]["landuse"] == "orchard" \
            or allFeatures["properties"]["landuse"] == "plant_nursery" or allFeatures["properties"]["landuse"] == "recreation_ground"\
            or allFeatures["properties"]["landuse"] == "village_green" or allFeatures["properties"]["landuse"] == "wasteland"\
            or allFeatures["properties"]["landuse"] == "farmland" or allFeatures["properties"]["landuse"] == "farmyard":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'outland'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["landuse"] == "commercial" or allFeatures["properties"]["landuse"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
   
            ## RESIDENTIAL        
            elif allFeatures["properties"]["landuse"] == "residential":
                landinfo = allFeatures['properties']
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo) 
        
        if "leisure" in allFeatures['properties']:
            ## OUTLAND(PARK)
            if allFeatures["properties"]["leisure"] == "park" or allFeatures["properties"]["leisure"] == "garden"\
            or allFeatures["properties"]["leisure"] == "pitch" or allFeatures["properties"]["leisure"] == "playground"\
            or allFeatures["properties"]["leisure"] == "recreation_ground":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'outland'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
        if "building" in allFeatures["properties"]:
            ## RESIDENTIAL
            if allFeatures["properties"]["building"] == "house" or allFeatures["properties"]["building"] == "apartments" \
            or allFeatures["properties"]["building"] == "residential" or allFeatures["properties"]["building"] == "detached"\
            or allFeatures["properties"]["building"] == "hotel":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'residential'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["building"] == "yes" or allFeatures["properties"]["building"] == "university" \
            or allFeatures["properties"]["building"] == "church" or allFeatures["properties"]["building"] == "office" \
            or allFeatures["properties"]["building"] == "museum" or allFeatures["properties"]["building"] == "school"\
            or allFeatures["properties"]["building"] == "commercial" or allFeatures["properties"]["building"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
osmLands = pd.DataFrame.from_dict(osmLands)
osmLands = osmLands.set_index('id')

In [42]:
# path = './data/geojson/unseen_central_london.geojson'
# Edinburgh
# only split into two classes: building (building + residential) & others

osmLands = []
for i in range(1,2):
    path = './data/geojson/trainingmap.geojson'
    with open(path, encoding="utf-8") as f:
        osmlines = geojson.load(f)
    for allFeatures in osmlines.features:
        if 'landuse' in allFeatures['properties']:
            ## OUTLAND
            if allFeatures["properties"]["landuse"] == "grass" or allFeatures["properties"]["landuse"] == "basin" \
            or allFeatures["properties"]["landuse"] == "forest" or allFeatures["properties"]["landuse"] == "greenfield" \
            or allFeatures["properties"]["landuse"] == "meadow" or allFeatures["properties"]["landuse"] == "orchard" \
            or allFeatures["properties"]["landuse"] == "plant_nursery" or allFeatures["properties"]["landuse"] == "recreation_ground"\
            or allFeatures["properties"]["landuse"] == "village_green" or allFeatures["properties"]["landuse"] == "wasteland"\
            or allFeatures["properties"]["landuse"] == "farmland" or allFeatures["properties"]["landuse"] == "farmyard":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'others'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["landuse"] == "commercial" or allFeatures["properties"]["landuse"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
   
            ## RESIDENTIAL        
            elif allFeatures["properties"]["landuse"] == "residential":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo) 
        
        if "leisure" in allFeatures['properties']:
            ## OUTLAND(PARK)
            if allFeatures["properties"]["leisure"] == "park" or allFeatures["properties"]["leisure"] == "garden"\
            or allFeatures["properties"]["leisure"] == "pitch" or allFeatures["properties"]["leisure"] == "playground"\
            or allFeatures["properties"]["leisure"] == "recreation_ground":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'others'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
        if "building" in allFeatures["properties"]:
            ## RESIDENTIAL
            if allFeatures["properties"]["building"] == "house" or allFeatures["properties"]["building"] == "apartments" \
            or allFeatures["properties"]["building"] == "residential" or allFeatures["properties"]["building"] == "detached"\
            or allFeatures["properties"]["building"] == "hotel":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["building"] == "yes" or allFeatures["properties"]["building"] == "university" \
            or allFeatures["properties"]["building"] == "church" or allFeatures["properties"]["building"] == "office" \
            or allFeatures["properties"]["building"] == "museum" or allFeatures["properties"]["building"] == "school"\
            or allFeatures["properties"]["building"] == "commercial" or allFeatures["properties"]["building"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
osmLands = pd.DataFrame.from_dict(osmLands)
osmLands = osmLands.set_index('id')

In [24]:
# path = './data/geojson/unseen_central_london.geojson'
# London / Delhi

osmLands = []
for i in range(1,6):
    path = './data/geojson/toby_new/map{}.geojson'.format(i) 
    with open(path, encoding="utf-8") as f:
        osmlines = geojson.load(f)
    for allFeatures in osmlines.features:
        if 'landuse' in allFeatures['properties']:
            ## OUTLAND
            if allFeatures["properties"]["landuse"] == "grass" or allFeatures["properties"]["landuse"] == "basin" \
            or allFeatures["properties"]["landuse"] == "forest" or allFeatures["properties"]["landuse"] == "greenfield" \
            or allFeatures["properties"]["landuse"] == "meadow" or allFeatures["properties"]["landuse"] == "orchard" \
            or allFeatures["properties"]["landuse"] == "plant_nursery" or allFeatures["properties"]["landuse"] == "recreation_ground"\
            or allFeatures["properties"]["landuse"] == "village_green" or allFeatures["properties"]["landuse"] == "wasteland"\
            or allFeatures["properties"]["landuse"] == "farmland" or allFeatures["properties"]["landuse"] == "farmyard":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'outland'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["landuse"] == "commercial" or allFeatures["properties"]["landuse"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
   
            ## RESIDENTIAL        
            elif allFeatures["properties"]["landuse"] == "residential":
                landinfo = allFeatures['properties']
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo) 
        
        if "leisure" in allFeatures['properties']:
            ## OUTLAND(PARK)
            if allFeatures["properties"]["leisure"] == "park" or allFeatures["properties"]["leisure"] == "garden"\
            or allFeatures["properties"]["leisure"] == "pitch" or allFeatures["properties"]["leisure"] == "playground"\
            or allFeatures["properties"]["leisure"] == "recreation_ground":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'outland'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
        if "building" in allFeatures["properties"]:
            ## RESIDENTIAL
            if allFeatures["properties"]["building"] == "house" or allFeatures["properties"]["building"] == "apartments" \
            or allFeatures["properties"]["building"] == "residential" or allFeatures["properties"]["building"] == "detached"\
            or allFeatures["properties"]["building"] == "hotel":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'residential'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["building"] == "yes" or allFeatures["properties"]["building"] == "university" \
            or allFeatures["properties"]["building"] == "church" or allFeatures["properties"]["building"] == "office" \
            or allFeatures["properties"]["building"] == "museum" or allFeatures["properties"]["building"] == "school"\
            or allFeatures["properties"]["building"] == "commercial" or allFeatures["properties"]["building"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
osmLands = pd.DataFrame.from_dict(osmLands)
osmLands = osmLands.set_index('osm_id')

In [80]:
# path = './data/geojson/unseen_central_london.geojson'
# London / Delhi
# only split into two classes: building (building + residential) & others

osmLands = []
for i in range(1,6):
    path = './data/geojson/map{}.geojson'.format(i) 
    with open(path, encoding="utf-8") as f:
        osmlines = geojson.load(f)
    for allFeatures in osmlines.features:
        if 'landuse' in allFeatures['properties']:
            ## OUTLAND
            if allFeatures["properties"]["landuse"] == "grass" or allFeatures["properties"]["landuse"] == "basin" \
            or allFeatures["properties"]["landuse"] == "forest" or allFeatures["properties"]["landuse"] == "greenfield" \
            or allFeatures["properties"]["landuse"] == "meadow" or allFeatures["properties"]["landuse"] == "orchard" \
            or allFeatures["properties"]["landuse"] == "plant_nursery" or allFeatures["properties"]["landuse"] == "recreation_ground"\
            or allFeatures["properties"]["landuse"] == "village_green" or allFeatures["properties"]["landuse"] == "wasteland"\
            or allFeatures["properties"]["landuse"] == "farmland" or allFeatures["properties"]["landuse"] == "farmyard":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'others'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["landuse"] == "commercial" or allFeatures["properties"]["landuse"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
   
            ## RESIDENTIAL        
            elif allFeatures["properties"]["landuse"] == "residential":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo) 
        
        if "leisure" in allFeatures['properties']:
            ## OUTLAND(PARK)
            if allFeatures["properties"]["leisure"] == "park" or allFeatures["properties"]["leisure"] == "garden"\
            or allFeatures["properties"]["leisure"] == "pitch" or allFeatures["properties"]["leisure"] == "playground"\
            or allFeatures["properties"]["leisure"] == "recreation_ground":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'others'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
        if "building" in allFeatures["properties"]:
            ## RESIDENTIAL
            if allFeatures["properties"]["building"] == "house" or allFeatures["properties"]["building"] == "apartments" \
            or allFeatures["properties"]["building"] == "residential" or allFeatures["properties"]["building"] == "detached"\
            or allFeatures["properties"]["building"] == "hotel":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
            
            ## BUILDING
            elif allFeatures["properties"]["building"] == "yes" or allFeatures["properties"]["building"] == "university" \
            or allFeatures["properties"]["building"] == "church" or allFeatures["properties"]["building"] == "office" \
            or allFeatures["properties"]["building"] == "museum" or allFeatures["properties"]["building"] == "school"\
            or allFeatures["properties"]["building"] == "commercial" or allFeatures["properties"]["building"] == "retail":
                landinfo = allFeatures['properties']
                landinfo["landuse"] = 'building'
                locarr = np.array(allFeatures['geometry']['coordinates'], dtype="object")
                if locarr.ndim > 2:
                    locarr = locarr.reshape(-1, locarr.shape[-1])  # 3-dim -> 2-dim        
                if locarr.ndim == 1:
                    locarr = np.array([locarr,locarr])        
                if locarr.shape[0] > 2:
                    landinfo['location'] = locarr
                    osmLands.append(landinfo)
                
osmLands = pd.DataFrame.from_dict(osmLands)
osmLands = osmLands.set_index('osm_id')

In [51]:
osmLands.head()

,name,landuse,location,locationPolygon
osm_id,,,,
1566802,Beaufort Mansions,building,"[[-0.175123, 51.483269], [-0.175059, 51.483295...","POLYGON ((-0.175123 51.483269, -0.175059 51.48..."
NaN,NaN,outland,"[[-0.090421, 51.66395], [-0.09184, 51.664016],...","POLYGON ((-0.090421 51.66395, -0.09184 51.6640..."
NaN,NaN,outland,"[[-0.086632, 51.662146], [-0.086531, 51.662931...","POLYGON ((-0.086632 51.662146, -0.086531 51.66..."
NaN,NaN,outland,"[[-0.082658, 51.661062], [-0.082272, 51.661765...","POLYGON ((-0.082658 51.661062, -0.082272 51.66..."
NaN,Bush Hill Park,outland,"[[-0.06909, 51.648324], [-0.069091, 51.64887],...","POLYGON ((-0.06909 51.648324, -0.069091 51.648..."


In [53]:
# Filtering and pre-processing 
#osmLands = osmLands[osmLands['type'] != 'multipolygon']
#osmLands['utmLocation'] = osmLands.apply(lambda row: utm_convert(row), axis=1) # convert to utm coordinate
osmLands['locationPolygon'] = osmLands.apply(lambda row: polygon(row), axis=1)
# construct the line using a list of coordinate-tuples
osmLands = osmLands.filter(['name','landuse', 'location', 'locationPolygon'])
# only care about related attributes 
np.unique(osmLands['landuse'].astype(str), return_counts=True)

(array(['building', 'outland', 'residential'], dtype=object),
 array([200181,  20826, 176064]))

In [54]:
osmLands.dropna(subset=['locationPolygon'], inplace=True)

In [55]:
osm_lands = gpd.GeoDataFrame(osmLands)
osm_lands = osm_lands.rename({'locationPolygon': 'geometry'}, axis = 1)
osm_lands['geometry'] = gpd.GeoSeries(osm_lands['geometry'])

## Add landuse type as a feature

First check whether data points are in the corresponding land

If the point is in a land, make the land type as a feature of this point, or add "null" feature (not in any land type)

In [ ]:
#data_path = os.path.join(data_dir, "new_data_set_with_station_roads_distance.csv")
#data_path = os.path.join(data_dir, "london_unseen_data_with_roads_distance.csv")
data_path = os.path.join(data_dir, "all_data_new_select_with_roads_distance.csv")
full_data = pd.read_csv(data_path, index_col=0)
full_data.head()

In [58]:
hits_full = bbox_full.apply(lambda row: list(osm_lands.sindex.intersection(row)), axis=1)

In [59]:
dist_df_full = pd.DataFrame({'pt_idx':np.repeat(hits_full.index, hits_full.apply(len)), 'close_land_idx': np.concatenate(hits_full.values)})
dist_df_full = dist_df_full.join(new_full_data['geometry'].rename('point'), on='pt_idx')
dist_df_full = dist_df_full.join(osm_lands[['geometry','location', 'landuse']].reset_index(drop=True), on='close_land_idx')

dist_gdf_full = gpd.GeoDataFrame(dist_df_full)
dist_gdf_full['if_contain'] = dist_gdf_full['geometry'].contains(gpd.GeoSeries(dist_gdf_full['point']))

dist_gdf_full.shape

(62554, 7)

In [ ]:
dist_gdf_full

In [61]:
contain = dist_gdf_full[dist_gdf_full['if_contain'] == True]
df_contain = contain[['pt_idx', 'landuse', 'close_land_idx']].set_index('pt_idx')
new_full_data = full_data.join(df_contain)
new_full_data['landuse'].fillna('others', inplace=True)
np.unique(new_full_data['landuse'].astype(str), return_counts=True)

(array(['building', 'others', 'outland', 'residential'], dtype=object),
 array([3168, 3911,  220,  421]))

In [62]:
#new_full_data.loc[new_full_data['i/o'].isin([2,1]), 'i/o'] = 1 # commuting 

In [63]:
#new_full_data.to_csv(os.path.join(data_dir,'new_data_set_with_station_roads_distance_land.csv'))
#new_full_data.to_csv(os.path.join(data_dir,'london_unseen_data_with_roads_distance_land.csv'))
new_full_data.to_csv(os.path.join(data_dir,'all_data_new_select_with_roads_distance_land.csv'))

In [66]:
new_full_data.columns

Index(['Unnamed: 0.1', 'phoneTimestamp', 'pm1', 'pm2_5', 'pm10', 'temperature',
       'humidity', 'bin0', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6',
       'bin7', 'bin8', 'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14',
       'bin15', 'total', 'gpsLongitude', 'gpsLatitude', 'gpsAltitude',
       'gpsAccuracy', 'luxLevel', 'motion', 'battery', 'subjectID', 'location',
       'activity', 'location_type', 'activity_type', 'recording_id',
       'timestamp', 'tubeline', 'notes', 'geometry', 'highway', 'distance',
       'close_road_idx', 'landuse', 'close_land_idx'],
      dtype='object')

## Example

In [554]:
color_index_road = {'footway' : 'crimson', 'motorway': 'orange', 'primary_secondary': 'gray', 'residential': 'green',
        'tertiary': 'orange', 'unclassified': 'black', 'unknown': 'darkblue' }

In [ ]:
m = folium.Map(full_data[['gpsLatitude', 'gpsLongitude']].iloc[0], zoom_start=16)
for i in range(len(osm_roads)):
    idx = i
    #print(osmRoads.iloc[idx])
    coords = [[x[1], x[0]] for x in osm_roads.iloc[idx]['location']]
    label = osm_roads.iloc[idx]['highway']
    c = color_index_road[label]
        
    line = folium.PolyLine(locations = coords, color = c)
    m.add_child(line)

In [569]:
m = folium.Map(new_full_data[['gpsLatitude', 'gpsLongitude']].iloc[0], zoom_start=16)
folium.Circle(location=[new_full_data['gpsLatitude'].iloc[0], new_full_data['gpsLongitude'].iloc[0]], radius=10, color='darkblue').add_to(m)

for i in range(len(osm_roads)):
    idx = i
    #print(osmRoads.iloc[idx])
    coords = [[x[1], x[0]] for x in osm_roads.iloc[idx]['location']]
    label = osm_roads.iloc[idx]['highway']
    c = 'green'
        
    line = folium.PolyLine(locations = coords, color = c)
    m.add_child(line)
    
folium.PolyLine([[x[1],x[0]] for x in dist_gdf_full.iloc[0]['location']], color = 'crimson').add_to(m)   


In [588]:
color_index = {'building' : 'crimson', 'outland': 'darkblue', 'others': 'green',
        'residential': 'orange'}

In [589]:
m = folium.Map(new_full_data[['gpsLatitude', 'gpsLongitude']].iloc[0], zoom_start=16)
for i in range(len(osm_lands)):
    idx = i
    #print(osmRoads.iloc[idx])
    coords = [[x[1], x[0]] for x in osm_lands.iloc[idx]['location']]
    label = osm_lands.iloc[idx]['landuse']
    c = color_index[label]
        
    line = folium.PolyLine(locations = coords, color = c)
    m.add_child(line)

In [593]:
m = folium.Map(new_full_data[['gpsLatitude', 'gpsLongitude']].iloc[2], zoom_start=16)
folium.Circle(location=[new_full_data['gpsLatitude'].iloc[2], new_full_data['gpsLongitude'].iloc[2]], radius=5, color='darkblue').add_to(m)

for i in range(len(osm_lands)):
    idx = i
    #print(osmRoads.iloc[idx])
    coords = [[x[1], x[0]] for x in osm_lands.iloc[idx]['location']]
    label = osm_lands.iloc[idx]['landuse']        
    line = folium.PolyLine(locations = coords, color = "green")
    m.add_child(line)

folium.PolyLine([[x[1],x[0]] for x in dist_gdf_full.iloc[2]['location']], color = 'crimson').add_to(m)   
